In [1]:
!nvidia-smi

Sun Feb 12 10:35:51 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    25W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os
from getpass import getpass

user = getpass("Github User:")
password = getpass("Github Password:")
os.environ['GITHUB_AUTH'] = user + ':' + password

Github User:··········
Github Password:··········


In [4]:
REPO_DIR = '/demo'
%mkdir -p "$REPO_DIR"
%cd "$REPO_DIR" 
!git clone https://$GITHUB_AUTH@github.com/srijarkoroy/LinkSeg 
%cd "$REPO_DIR/LinkSeg"

/demo
Cloning into 'LinkSeg'...
remote: Enumerating objects: 86, done.
remote: Counting objects: 100% (86/86), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 86 (delta 33), reused 64 (delta 16), pack-reused 0
Unpacking objects: 100% (86/86), 307.10 KiB | 8.77 MiB/s, done.
/demo/LinkSeg


In [5]:
!unzip /content/drive/MyDrive/datasets.zip -d /demo/LinkSeg

Archive:  /content/drive/MyDrive/datasets.zip
mapname:  conversion of  failed
 extracting: /demo/LinkSeg/test.zip  
 extracting: /demo/LinkSeg/training.zip  


In [6]:
%mkdir dataset/

In [7]:
!unzip /demo/LinkSeg/test.zip -d /demo/LinkSeg/dataset/

Archive:  /demo/LinkSeg/test.zip
   creating: /demo/LinkSeg/dataset/test/
   creating: /demo/LinkSeg/dataset/test/images/
  inflating: /demo/LinkSeg/dataset/test/images/01_test.tif  
  inflating: /demo/LinkSeg/dataset/test/images/02_test.tif  
  inflating: /demo/LinkSeg/dataset/test/images/03_test.tif  
  inflating: /demo/LinkSeg/dataset/test/images/04_test.tif  
  inflating: /demo/LinkSeg/dataset/test/images/05_test.tif  
  inflating: /demo/LinkSeg/dataset/test/images/06_test.tif  
  inflating: /demo/LinkSeg/dataset/test/images/07_test.tif  
  inflating: /demo/LinkSeg/dataset/test/images/08_test.tif  
  inflating: /demo/LinkSeg/dataset/test/images/09_test.tif  
  inflating: /demo/LinkSeg/dataset/test/images/10_test.tif  
  inflating: /demo/LinkSeg/dataset/test/images/11_test.tif  
  inflating: /demo/LinkSeg/dataset/test/images/12_test.tif  
  inflating: /demo/LinkSeg/dataset/test/images/13_test.tif  
  inflating: /demo/LinkSeg/dataset/test/images/14_test.tif  
  inflating: /demo/LinkS

In [8]:
!unzip /demo/LinkSeg/training.zip -d /demo/LinkSeg/dataset/

Archive:  /demo/LinkSeg/training.zip
   creating: /demo/LinkSeg/dataset/training/
   creating: /demo/LinkSeg/dataset/training/1st_manual/
 extracting: /demo/LinkSeg/dataset/training/1st_manual/21_manual1.gif  
 extracting: /demo/LinkSeg/dataset/training/1st_manual/22_manual1.gif  
 extracting: /demo/LinkSeg/dataset/training/1st_manual/23_manual1.gif  
 extracting: /demo/LinkSeg/dataset/training/1st_manual/24_manual1.gif  
 extracting: /demo/LinkSeg/dataset/training/1st_manual/25_manual1.gif  
 extracting: /demo/LinkSeg/dataset/training/1st_manual/26_manual1.gif  
 extracting: /demo/LinkSeg/dataset/training/1st_manual/27_manual1.gif  
 extracting: /demo/LinkSeg/dataset/training/1st_manual/28_manual1.gif  
 extracting: /demo/LinkSeg/dataset/training/1st_manual/29_manual1.gif  
 extracting: /demo/LinkSeg/dataset/training/1st_manual/30_manual1.gif  
 extracting: /demo/LinkSeg/dataset/training/1st_manual/31_manual1.gif  
 extracting: /demo/LinkSeg/dataset/training/1st_manual/32_manual1.gif 

In [9]:
data_path = "/demo/LinkSeg/dataset"
print(data_path)

/demo/LinkSeg/dataset


In [10]:
!cat data/augment.py

import os
import numpy as np
import cv2
from glob import glob
from tqdm import tqdm

import imageio
from albumentations import HorizontalFlip, VerticalFlip, Rotate


def create_dir(path):

    ### Create a directory
    if not os.path.exists(path):
        os.makedirs(path)

def load_data(path):

    ### Split data into train and test set
    train_x = sorted(glob(os.path.join(path, "training", "images", "*.tif")))
    train_y = sorted(glob(os.path.join(path, "training", "1st_manual", "*.gif")))

    test_x = sorted(glob(os.path.join(path, "test", "images", "*.tif")))
    test_y = sorted(glob(os.path.join(path, "test", "mask", "*.gif")))

    return (train_x, train_y), (test_x, test_y)

def augment_data(images, masks, save_path, augment=True):

    ### Augmenting data into Horizontal,Vertical & Rotate

    size = (512, 512)

    for idx, (x, y) in tqdm(enumerate(zip(images, masks)), total=len(images)):
        """ Extracting the name """
        name = x.split("/")[-1].split(".")[0]

 

In [11]:
from data.augment import load_data
(train_x, train_y), (test_x, test_y) = load_data(data_path)

In [12]:
print(f"Train: {len(train_x)} - {len(train_y)}")
print(f"Test: {len(test_x)} - {len(test_y)}")

Train: 20 - 20
Test: 20 - 20


In [13]:
from data.augment import create_dir

create_dir("/demo/LinkSeg/new_data/train/image")
create_dir("/demo/LinkSeg/new_data/train/mask")
create_dir("/demo/LinkSeg/new_data/test/image/")
create_dir("/demo/LinkSeg/new_data/test/mask/")

In [14]:
from data.augment import augment_data

augment_data(train_x, train_y, "/demo/LinkSeg/new_data/train", augment=True)
augment_data(test_x, test_y, "/demo/LinkSeg/new_data/test", augment=False)

100%|██████████| 20/20 [00:00<00:00, 36.43it/s]


In [15]:
import torch
from torch.utils.data import DataLoader

import os
from glob import glob

from data.augment import load_data
from data.dataset import DriveDataset


data_path = "/demo/LinkSeg/new_data"

train_x = sorted(glob(os.path.join(data_path, "train", "image", "*.png")))
train_y = sorted(glob(os.path.join(data_path, "train", "mask", "*.png")))

valid_x = sorted(glob(os.path.join(data_path, "test", "image", "*.png")))
valid_y = sorted(glob(os.path.join(data_path, "test", "mask", "*.png")))

print(f"Train: {len(train_x)} - {len(train_y)}")
print(f"Test: {len(valid_x)} - {len(valid_y)}")

train_dataset = DriveDataset(train_x, train_y)
 
train_loader = DataLoader(
        dataset=train_dataset,
        batch_size=4,
        shuffle=True,
        num_workers=2
    )

val_dataset = DriveDataset(valid_x, valid_y)

val_loader = DataLoader(
        dataset=val_dataset,
        batch_size=4,
        shuffle=False,
        num_workers=2
    )

Train: 80 - 80
Test: 20 - 20


In [16]:
!cat linkseg/__init__.py

from .loss import DiceLoss, IoU
from .linknet import LinkNet
from .training_utils import Train, Evaluate

__all__ = [
    DiceLoss,
    IoU,
    LinkNet,
    Train,
    Evaluate,
]

In [17]:
from linkseg import LinkNet, DiceLoss, IoU, Train, Evaluate

from tqdm import tqdm

# Training and Evaluate object
train = Train(dice=DiceLoss(), iou=IoU())
eval = Evaluate(dice=DiceLoss(), iou=IoU())

# Model Initialization and setting up hyperparameters
model = LinkNet()
print(model)

# Hyperparameters
lr = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
epochs = 50

# Training
for epoch in tqdm(range(epochs)):
    print("Epoch: ", epoch)

    train_dice, train_iou = train.forward(model=model, loader=train_loader, optimizer=optimizer)
    val_dice, val_iou = eval.forward(model=model, loader=val_loader)

torch.save(model.state_dict(), 'linknet.pth')

LinkNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (encoder1): Encoder(
    (block1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (block2): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplac

  0%|          | 0/50 [00:00<?, ?it/s]

Epoch:  0
Shifting the model to cuda!
Train Dice Loss: 0.8504205197095871,  Train IoU Loss: 0.9191329091787338, 
Shifting the model to cuda!


  2%|▏         | 1/50 [00:15<12:37, 15.47s/it]


Validation Dice Loss: 0.4209925174713135,  Validation IoU Loss: 0.5925327420234681, 
Epoch:  1
Shifting the model to cuda!
Train Dice Loss: 0.8503723978996277,  Train IoU Loss: 0.9191176295280457, 
Shifting the model to cuda!


...


Validation Dice Loss: 0.4209925174713135,  Validation IoU Loss: 0.5925327420234681, 
Epoch:  48
Shifting the model to cuda!
Train Dice Loss: 0.8503808617591858,  Train IoU Loss: 0.9191202849149704, 
Shifting the model to cuda!


 98%|█████████▊| 49/50 [03:13<00:03,  3.71s/it]


Validation Dice Loss: 0.4209925174713135,  Validation IoU Loss: 0.5925327420234681, 
Epoch:  49
Shifting the model to cuda!
Train Dice Loss: 0.8504098802804947,  Train IoU Loss: 0.9191293507814408, 
Shifting the model to cuda!


100%|██████████| 50/50 [03:17<00:00,  3.95s/it]


Validation Dice Loss: 0.4209925174713135,  Validation IoU Loss: 0.5925327420234681, 


In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/linknet.pth')